In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [42]:
def robustScaler(arr):
    p1 = np.nanpercentile(arr, 5)
    p99 = np.nanpercentile(arr, 95)
    out = (arr - p1) / (p99 - p1)
    out -= np.nanmin(out)
    out[np.where(out>1)] = 1
    return out

In [60]:
dataset = pd.read_csv('datasets/TOI_2025.11.13_04.26.29.csv', comment='#', skiprows=lambda x: x < 4)
dataset['tfopwg_disp'] = dataset['tfopwg_disp'].replace({'KP': 'P', 'CP': 'P', 'FA': 'F', 'FP': 'F'})
dataset = dataset.rename(columns={'tfopwg_disp': 'isExoplanet'})

In [61]:
for i in ['rastr','decstr','toi_created','rowupdate']:
    dataset.drop(i,axis=1,inplace=True)
for i in dataset.keys():
    try:
        dataset[i] = np.array(robustScaler(np.array(dataset[i])))
    except:
        print(i)
nanCount = dataset.isna().sum()
for i in nanCount.keys():
    if nanCount[i] > 5000:
        dataset.drop(i,axis=1,inplace=True)

isExoplanet


/tmp/ipykernel_170622/741473098.py:4: RuntimeWarning: invalid value encountered in divide
  out = (arr - p1) / (p99 - p1)
/tmp/ipykernel_170622/741473098.py:5: RuntimeWarning: All-NaN slice encountered
  out -= np.nanmin(out)
/home/thomas/anaconda3/envs/brain/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1409: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
